In [3]:
import xml.etree.ElementTree
import os,glob,sys
import pandas as pd
from sklearn.decomposition import PCA
import pylab as plt
import seaborn as sns
import numpy as np
import pickle

%load_ext autoreload
%autoreload 2

sys.path.append("..")
from mzmine import pick_peaks, align, match_aligned_to_original, load_aligned_peaks
import alignment
import ms2_matching

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
WIN = 'D:/'
#MAC = '/Users/anamaria/Documents/'
MAC = '/Volumes/Transcend2/'

osp = WIN

In [5]:
filtered_peaks = pd.read_csv(osp+'pymz/mzmine/221119/modifiedRTdatafiltered.txt', sep = '\t', index_col = 0)
filtered_peaks_ids = list(filtered_peaks['custom_id'])

In [6]:
with open(os.path.join(osp,'pymz/mzmine/aligner_gp.txt'), 'rb') as file:  
    aligner_gp = pickle.load(file)

In [8]:
import metab_utils
filtered_peaksets = metab_utils.extract_peaksets_with_spectra_from_filt_dataset(aligner_gp, filtered_peaks_ids)

In [31]:
def makeClusterList(dataset, peaksets):
    from mnet import Cluster
    clusters = []
    for peakset in filtered_peaksets:
        for peak in peakset.peaks:
            if dataset in peak.source_file.lower():
                new_cl = Cluster(peak.ms2_spectrum, peak.source_id)
                clusters.append(new_cl)
    return clusters

In [32]:
mal_mgf_file = os.path.join(osp, 'pymz/mzmine/210120/mal_filtered.mgf')
mal_clusters = makeClusterList('mal', filtered_peaksets)

In [37]:
zik_mgf_file = os.path.join(osp, 'pymz/mzmine/210120/zik_filtered.mgf')
zik_clusters = makeClusterList('zik', filtered_peaksets)

In [38]:
with open(zik_mgf_file, 'w') as f:
    for cl in zik_clusters:
        f.write(cl.get_mgf_string())